# good reads

In [ ]:
# import what we need/didnt already have
!pip3 install goodreads_api_client isbnlib newspaper3k tqdm progressbar scipy 

In [ ]:
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import isbnlib
from newspaper import Article
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm
from progressbar import ProgressBar
import re
import copy
from scipy import stats
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import goodreads_api_client as gr
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA


### lets see what we're working with

In [ ]:
df = pd.read_csv('../input/goodreadsbooks/books.csv', error_bad_lines = False)

In [ ]:
df.index = df['bookID']

In [ ]:
#and what 
df.head()

In [ ]:
print("Dataset contains {} rows and {} columns".format(df.shape[0], df.shape[1]))

In [ ]:
#apparently no Nulls!
df["average_rating"].isnull().value_counts()

In [ ]:
#drop NaNs anyway
df.dropna(0, inplace=True)

Columns Description:
* **bookID** Contains the unique ID for each book/series
* **title** contains the titles of the books
* **authors** contains the author of the particular book
* **average_rating** the average rating of the books, as decided by the users
* **ISBN** ISBN(10) number, tells the information about a book - such as edition and publisher
* **ISBN 13** The new format for ISBN, implemented in 2007. 13 digits
* **language_code** Tells the language for the books
* **Num_pages** Contains the number of pages for the book
* **Ratings_count** Contains the number of ratings given for the book
* **text_reviews_count** Has the count of reviews left by users

In [ ]:
df.columns

* Dropping all books with 30 or less pages.... 
* dont BS me with "yea i reaad 50+ books/yr" if u read infant books.
* also there are like 200ish audio books/that have just a title page or publisher page etc etc
* and like 20 books with 0 pages.. somehow?
* akso lets down books with 2000+ pages. 
* if ur planning on reading east of eden its not like ur gonna take the rec from a machine

In [ ]:
df = df[df['  num_pages'] >= 30]
df.shape

In [ ]:
z = np.abs(stats.zscore(df['  num_pages']))
for i in range(len(list(np.where(z>=3)[0]))):
    df.drop(df.index[np.where(z>=3)[0][i]], inplace=True)
df.shape

### processing data

Let's scale our numerical data for PCA:

In [ ]:
ss = StandardScaler()
df2 = ss.fit_transform(df[['average_rating', 'ratings_count', 'text_reviews_count']])

In [ ]:
pca = PCA()
pca_data = pca.fit_transform(df2)
new_df = pd.DataFrame(pca_data)
new_df

In [ ]:
#in the range of 2-30, lets find the best n_clusters that best explain our books.# 
X = new_df
distortions = []
for k in range(2,30):
    k_means = KMeans(n_clusters = k)
    k_means.fit(X)
    distortions.append(k_means.inertia_)

fig = plt.figure(figsize=(15,10))
plt.plot(range(2,30), distortions, 'bx-')
plt.title("Elbow Curve")

 this is roughly "Genre clusters" ; eg Sci-Fi, Romance, Thriller, etc

In [ ]:
CENTROIDS = 7 

In [ ]:
k_means = KMeans(n_clusters = CENTROIDS)
new_df['idx'] = k_means.fit_predict(new_df)
new_df

using 7 as our k, we predict what category each book falls into. As you can see the first two rows are the same category, which makes sense because they are both harry potter books

### visualization of what we have so far

In [ ]:
ax3 = plt.axes(projection='3d')
ax3.scatter(new_df[0], new_df[1], new_df[2], edgecolor='black', c=new_df['idx'])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('plotting the 3 principle components')

In [ ]:
plt.scatter(new_df[0], new_df[1], edgecolor='black', c=new_df['idx'])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('plotting 2 principle components')

In [ ]:
plt.scatter(new_df[0], new_df[2], edgecolor='black', c=new_df['idx'])
plt.xlabel('PC1')
plt.ylabel('PC3')
plt.title('plotting 2 principle components')

In [ ]:
plt.scatter(new_df[1], new_df[2], edgecolor='black', c=new_df['idx'])
plt.xlabel('PC2')
plt.ylabel('PC3')
plt.title('plotting 2 principle components')

## ok outliers r being annoying 

In [ ]:
z = np.abs(stats.zscore(new_df[[0,1,2]]))
to_drop = np.unique(np.where(z>=3)[0])
print(to_drop, type(to_drop))

In [ ]:
new_df.drop(to_drop, inplace=True)
new_df

In [ ]:
# Set up the axes with gridspec
fig = plt.figure(figsize=(8, 8))
grid = plt.GridSpec(3, 3, hspace=0.2, wspace=0.2)
fig3 = plt.figure()
ax3 = Axes3D(fig3)

pc1_2 = fig.add_subplot(grid[:-1, 1:])
pc0_1 = fig.add_subplot(grid[:-1, 0], xticklabels=[], sharex=pc1_2)
pc0_2 = fig.add_subplot(grid[-1, 1:], yticklabels=[], sharey=pc1_2)

# scatter points on the main axes
ax3.scatter(new_df[0], new_df[1], new_df[2], edgecolor='black', c=new_df['idx'])

# PCs on the attached axes
pc0_1.scatter(new_df[0], new_df[1], edgecolor='black', c=new_df['idx'])
pc0_2.scatter(new_df[0], new_df[2], edgecolor='black', c=new_df['idx'])
pc1_2.scatter(new_df[1], new_df[2], edgecolor='black', c=new_df['idx'])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('plotting principle components')

# ML tiem

In [ ]:
#model.predict(42)

### lol u THOUGHT
### we gotta make dummys merge first

In [ ]:
#i was trying to put them back together but was getting some nasty NaNs.. 
#i had forgotten that i had dropped many columns
df.shape, new_df.shape
#see this is a problem, time to M E R G E

In [ ]:
df3 = df.copy()
#dropping all the outliers we found in the 2nd z score test for PCs (this took like way too long to figure out)
df3.drop(df3.index[to_drop], inplace=True)
df3.head(2)

In [ ]:
df3.shape, new_df.shape
#WOOOOO HOOOO OMG took 7h27 but u figured it out! im proud of past vince for tenacity!

In [ ]:
#no NaNs
df3.isnull().values.any(), new_df.isnull().values.any()

In [ ]:
for col in df3:
    print(col, df3.isnull()[col].value_counts(), "\n")

In [ ]:
#reseting index since we had dropped outliers before
#this may give an error, in which case comment out line 3
df3 = df3.drop('bookID', axis=1)
df3 = df3.reset_index()
df3 = df3.drop('index', axis=1)
df3

In [ ]:
#reseting index since we had dropped outliers before
new_df = new_df.reset_index()
new_df = new_df.drop("index", axis=1)
new_df

In [ ]:
#finally merging the two (PCA'd values with the original df)
df4 = df3.merge(new_df, right_index=True, left_index=True)
df4

In [ ]:
#reverifying there are no NaNs
for col in df4:
    print(col, df4.isnull()[col].value_counts(), "\n")

### creating the dummys

In [ ]:
#adding a new col 'rating_between' for future dummy
for i in range(5):
    df4.loc[ (df4['average_rating'] >= i) & (df4['average_rating'] <= i+1), 'rating_between'] = f"between {i} and {i+1}"

In [ ]:
#making a dummy col for each rating level
dummy_rating_df = df4['rating_between'].str.get_dummies(sep=",")
dummy_rating_df.head(3)

In [ ]:
#making a dummy col for each language
dummy_language_df = df4['language_code'].str.get_dummies(sep=",")
dummy_language_df.head(3)

### weighted ratings instead of actual ratings !!

![Weighted Rating calculation](https://miro.medium.com/max/736/1*fGziZl2Do-VyQXSCPq_Y2Q.png)

In [ ]:
def weighted_rating(df):
    v = df['ratings_count']
    R = df['average_rating']
    C = df['average_rating'].mean()
    m = df['ratings_count'].quantile(0.9)
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
#readjusting the weight of "average rating" to be proportional to the quantile of ratings count
df4['weighted_score'] = weighted_rating(df4)

In [ ]:
#renaming columns
df4.columns = ['bookID','title','authors','average_rating','isbn','isbn13','language_code',
               '  num_pages','ratings_count','text_reviews_count','publication_date',
               'publisher',"PC1","PC2","PC3",'idx','rating_between','weighted_score']
df4

## make a model df for the model

In [ ]:
model_df = pd.concat([dummy_rating_df, dummy_language_df, df4['weighted_score'], df4['average_rating'], df3['ratings_count'], df4['PC1'], df4['PC2'], df4['PC3']], axis=1)
model_df

In [ ]:
min_max_scaler = MinMaxScaler()
model_df_final = min_max_scaler.fit_transform(model_df)
model_df_final_kNN = model_df_final.copy()
model_df_final_DBSCAN = model_df_final.copy()
model_df_final_TSNE = model_df_final.copy()

# KNN

In [ ]:
knn_model = neighbors.NearestNeighbors(n_neighbors=CENTROIDS, algorithm='kd_tree')
knn_model.fit(model_df_final_kNN)
distances, indices = knn_model.kneighbors(model_df_final_kNN)

In [ ]:
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

# DBSCAN

In [ ]:
DBSCAN_model = DBSCAN(eps=0.2, min_samples=4, algorithm='kd_tree')
DBSCAN_model.fit(model_df_final_DBSCAN)

In [ ]:
model_df_final_DBSCAN

In [ ]:
#looks like unsupervised ML decides there are 21 genres
clusters = DBSCAN_model.labels_
np.unique(clusters)

In [ ]:
colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 
          'olive', 'goldenrod', 'lightcyan', 'navy', 'red', 'orange', 'violet',
           'green', 'yellow', 'cyan', 'magenta', 'black', 'white', 'blue', 'steelblue']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [ ]:
plt.scatter(model_df_final_DBSCAN[:,-1], model_df_final_DBSCAN[:,-2], c=vectorizer(clusters))

# TSNE

In [ ]:
TSNE_model = TSNE(n_components=2, learning_rate=150.0, n_iter = 690, verbose=1)
X_embedded = TSNE_model.fit_transform(model_df_final_TSNE)

In [ ]:
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], legend='full', palette='Spectral')

# recommend me a b00k

In [ ]:
df4.head(40)

In [ ]:
#use whatever model u want in place of df4:
#model_df_final_kNN 
#model_df_final_DBSCAN 
#model_df_final_TSNE 

all_books_names = list(df4.title.values)

In [ ]:
#generously 'forked' from https://www.kaggle.com/hoshi7/goodreads-analysis-and-recommending-books
def get_index_from_name(name):
    print("Your book recs for ", name)
    return df4[df4["title"]==name].index.tolist()[0]


def get_id_from_partial_name(partial):
    potential_books = dict()
    for name in all_books_names:
        if partial in name:
            potential_books[name] = all_books_names.index(name)
    
    print('u might have meant: [')
    for key in potential_books:
        print(key, potential_books[key], ',')
    print('] \n just for fun, lets use ', key, '\n' )
    
    return(potential_books[key])
            
def book_rec(query=None,indx=None):
    if indx:
        for index in indices[indx][1:]:
            print(df4.iloc[index]["title"])
    if query:
        try:
            found_id = get_index_from_name(query)
            for index in indices[found_id][1:]:
                print(df4.iloc[index]["title"])
        except:
            print("couldnt find. pls try another book")
            found_id = get_id_from_partial_name(query)
            for index in indices[found_id][1:]:
                print(df4.iloc[index]["title"])

In [ ]:
book_rec("Poor People")

In [ ]:
book_rec("Harry Potter")

this is where u can add what book u want (assuming its in the DB of 10405 entires)

In [ ]:
book_rec("YOUR BOOK HERE")